In [23]:
import os

In [24]:
import sys 
from utils.config_loaders import load_config

In [25]:
from dotenv import load_dotenv

In [26]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

In [27]:
#logger=CustomLogger().get_logger("exception_experiment")

In [28]:
# Load configuration
from logger.custom_logger import CustomLogger
from exception.custom_exception import DocumentPortalException

In [29]:
log=CustomLogger().get_logger("demo")

In [30]:
class ModelLoader:
    """
    A utility class to load embedding models and LLM models.
    """
    def __init__(self):
        load_dotenv()  # Loads environment variables from .env file.
        self._validate_env()  # Checks if needed keys are set.
        self.config = load_config()  # Loads settings from a config file.
        log.info("Configuration loaded successfully", config_keys=list(self.config.keys()))  # Logs which settings were loaded.

    def _validate_env(self):
        """
        Validate necessary environment variables.
        Ensure API keys exist.
        """
        required_vars = ["GOOGLE_API_KEY", "GROQ_API_KEY"]  # List of needed keys.
        self.api_keys = {key: os.getenv(key) for key in required_vars}  # Gets their values.
        missing = [k for k, v in self.api_keys.items() if not v]  # Finds any missing keys.
        if missing:
            log.error("Missing environment variables", missing_vars=missing)  # Logs error if missing.
            raise DocumentPortalException("Missing environment variables", sys)  # Stops program if missing.
        log.info("Environment variables validated", available_keys=[k for k in self.api_keys if self.api_keys[k]])  # Logs success if all present.

    def load_embeddings(self):
        """
        Load and return the embedding model.
        """
        try:
            log.info("Loading embedding model...")
            model_name = self.config["embedding_model"]["model_name"]  # Gets model name from config.
            return GoogleGenerativeAIEmbeddings(model=model_name)  # Loads embedding model.
        except Exception as e:
            log.error("Error loading embedding model", error=str(e))  # Logs error if loading fails.
            raise DocumentPortalException("Failed to load embedding model", sys)  # Stops program if loading fails.
    
    def load_llm(self):
        """
        Load and return the LLM model.
        """
        """Load LLM dynamically based on provider in config."""
        llm_block = self.config["llm"]  # Gets LLM settings from config.
        log.info("Loading LLM...")
        provider_key = os.getenv("LLM_PROVIDER", "groq")  # Uses provider from environment, defaults to "groq".
        if provider_key not in llm_block:
            log.error("LLM provider not found in config", provider_key=provider_key)
            raise ValueError(f"Provider '{provider_key}' not found in config")
        llm_config = llm_block[provider_key]  # Gets provider settings.
        provider = llm_config.get("provider")
        model_name = llm_config.get("model_name")
        temperature = llm_config.get("temperature", 0.2)
        max_tokens = llm_config.get("max_output_tokens", 2048)
        log.info("Loading LLM", provider=provider, model=model_name, temperature=temperature, max_tokens=max_tokens)
        if provider == "google":
            llm = ChatGoogleGenerativeAI(
                model=model_name,
                temperature=temperature,
                max_output_tokens=max_tokens
            )
            return llm
        elif provider == "groq":
            llm = ChatGroq(
                model=model_name,
                api_key=self.api_keys["GROQ_API_KEY"],  # Uses GROQ API key.
                temperature=temperature,
            )
            return llm
        # elif provider == "openai":
        #     return ChatOpenAI(
        #         model=model_name,
        #         api_key=self.api_keys["OPENAI_API_KEY"],
        #         temperature=temperature,
        #         max_tokens=max_tokens
        #     )
        else:
            log.error("Unsupported LLM provider", provider=provider)
            raise ValueError(f"Unsupported LLM provider: {provider}")



In [32]:
if __name__ == "__main__":
    loader = ModelLoader()  # Makes a ModelLoader object.
    embeddings = loader.load_embeddings()  # Loads embedding model.
    print(f"Embedding Model Loaded: {embeddings}")  # Prints embedding model.
    result = embeddings.embed_query("Hello, how are you?")  # Gets embedding for a sample text.
    print(f"Embedding Result: {result}")  # Prints embedding result.
    llm = loader.load_llm()  # Loads LLM model.
    print(f"LLM Loaded: {llm}")  # Prints LLM model.
    result = llm.invoke("Hello, how are you?")  # Gets LLM response for a sample text.
    print(f"LLM Result: {result.content}")  # Prints LLM response.

{"available_keys": ["GOOGLE_API_KEY", "GROQ_API_KEY"], "timestamp": "2025-08-06T18:30:03.249471Z", "level": "info", "event": "Environment variables validated"}


FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'

In [33]:
with open(r"D:\DocumentPortal\config\config.yaml", "r") as f:
    print(f.read())

faiss_db:
  collection_name: "document_portal"


embedding_model:
  provider: "google"
  model_name: "models/text-embedding-004"

retriever:
  top_k: 10

llm:
  groq:
    provider: "groq"
    model_name: "deepseek-r1-distill-llama-70b"
    temperature: 0
    max_output_tokens: 2048

  google:
    provider: "google"
    model_name: "gemini-2.0-flash"
    temperature: 0
    max_output_tokens: 2048
